# Arbitron database connectivity test

to reset Jupyter theme type "jt -r" in command line
(see https://github.com/dunovank/jupyter-themes)

In [1]:
from pprint import pprint
import pandas as pd
import numpy as np
from datetime import datetime
from markets.markets import Markets
from database.database import Database
from settings.settings import Settings

In [2]:
db = Database()

Connecting to database...OK


In [3]:
db.get_exchanges()
db.get_tokens()
db.get_pairs()

In [4]:
df_exchangs = db.df_exchanges
df_tokens = db.df_tokens
df_pairs = db.df_pairs[['pair', 'exchange']]

In [5]:
markets = Markets(db)

CCXT version: 1.17.27


In [6]:
#df_pairs.sort_values(['pair','exchange']).set_index('pair')
pairs = pd.DataFrame(df_pairs.groupby(['pair'])['exchange'].apply(list)).exchange.to_dict()

In [7]:
pairs

{'ADA/BTC': ['binance', 'bittrex', 'hitbtc2', 'cryptopia'],
 'ADA/DOGE': ['cryptopia'],
 'ADA/ETH': ['binance', 'bittrex', 'hitbtc2'],
 'ADA/LTC': ['cryptopia'],
 'ADA/USDT': ['binance', 'bittrex', 'hitbtc2'],
 'BCH/BTC': ['kraken',
  'poloniex',
  'yobit',
  'binance',
  'okex',
  'bittrex',
  'exmo',
  'hitbtc2',
  'cryptopia'],
 'BCH/DOGE': ['yobit', 'cryptopia'],
 'BCH/ETH': ['poloniex',
  'yobit',
  'binance',
  'okex',
  'bittrex',
  'exmo',
  'hitbtc2'],
 'BCH/LTC': ['cryptopia'],
 'BCH/USD': ['kraken', 'yobit', 'okex', 'exmo'],
 'BCH/USDT': ['poloniex',
  'binance',
  'okex',
  'bittrex',
  'hitbtc2',
  'cryptopia'],
 'BTC/USD': ['kraken', 'yobit', 'okex', 'bittrex', 'exmo'],
 'BTC/USDT': ['poloniex',
  'binance',
  'okex',
  'bittrex',
  'exmo',
  'hitbtc2',
  'cryptopia'],
 'BTG/BTC': ['yobit', 'binance', 'okex', 'bittrex', 'exmo', 'hitbtc2'],
 'BTG/DOGE': ['yobit'],
 'BTG/ETH': ['yobit', 'binance', 'bittrex', 'hitbtc2'],
 'BTG/USD': ['yobit', 'okex', 'exmo'],
 'BTG/USDT': ['

In [8]:
exchanges = df_pairs.groupby(['exchange'])['pair'].apply(list).to_dict()

In [9]:
exchanges

{'binance': ['ADA/BTC',
  'ADA/ETH',
  'ADA/USDT',
  'BCH/BTC',
  'BCH/ETH',
  'BCH/USDT',
  'BTC/USDT',
  'BTG/BTC',
  'BTG/ETH',
  'DASH/BTC',
  'DASH/ETH',
  'EOS/BTC',
  'EOS/ETH',
  'EOS/USDT',
  'ETC/BTC',
  'ETC/ETH',
  'ETC/USDT',
  'ETH/BTC',
  'ETH/USDT',
  'ICX/BTC',
  'ICX/ETH',
  'ICX/USDT',
  'LTC/BTC',
  'LTC/ETH',
  'LTC/USDT',
  'NEO/BTC',
  'NEO/ETH',
  'NEO/USDT',
  'OMG/BTC',
  'OMG/ETH',
  'SC/BTC',
  'SC/ETH',
  'TRX/BTC',
  'TRX/ETH',
  'TRX/USDT',
  'XEM/BTC',
  'XEM/ETH',
  'XLM/BTC',
  'XLM/ETH',
  'XLM/USDT',
  'XMR/BTC',
  'XMR/ETH',
  'XRP/BTC',
  'XRP/ETH',
  'XRP/USDT',
  'XVG/BTC',
  'XVG/ETH',
  'ZEC/BTC',
  'ZEC/ETH'],
 'bittrex': ['ADA/BTC',
  'ADA/ETH',
  'ADA/USDT',
  'BCH/BTC',
  'BCH/ETH',
  'BCH/USDT',
  'BTC/USD',
  'BTC/USDT',
  'BTG/BTC',
  'BTG/ETH',
  'BTG/USDT',
  'DASH/BTC',
  'DASH/ETH',
  'DASH/USDT',
  'DCR/BTC',
  'DCR/USDT',
  'DOGE/BTC',
  'ETC/BTC',
  'ETC/ETH',
  'ETC/USDT',
  'ETH/BTC',
  'ETH/USD',
  'ETH/USDT',
  'LTC/BTC',
  'L

In [10]:
exchanges['hitbtc2'][3:5]

['BCH/BTC', 'BCH/ETH']

In [11]:
print(exchanges['exmo'])

['BCH/BTC', 'BCH/ETH', 'BCH/USD', 'BTC/USD', 'BTC/USDT', 'BTG/BTC', 'BTG/USD', 'DASH/BTC', 'DASH/USD', 'DOGE/BTC', 'EOS/BTC', 'EOS/USD', 'ETC/BTC', 'ETC/USD', 'ETH/BTC', 'ETH/LTC', 'ETH/USD', 'ETH/USDT', 'LTC/BTC', 'LTC/USD', 'USDT/USD', 'XLM/BTC', 'XLM/USD', 'XMR/BTC', 'XMR/USD', 'XRP/BTC', 'XRP/USD', 'ZEC/BTC', 'ZEC/USD']


# Get last access time

In [12]:
exchange = "exmo"
pair = "BCH/BTC"
cql = f"""SELECT ts FROM arbitron.history WHERE exchange='{exchange}' and pair='{pair}' LIMIT 1"""
cql

"SELECT ts FROM arbitron.history WHERE exchange='exmo' and pair='BCH/BTC' LIMIT 1"

In [13]:
res = db.session.execute(cql, timeout=5)
df = res._current_rows

In [14]:
int(df.iloc[0].values[0].astype(np.uint64)/1e6)

1532695029000

## Cool C* PER PARTITION feature for fetching last access times in single pass

In [15]:
cql = "select * from arbitron.history per partition limit 1"

In [16]:
res = db.session.execute(cql, timeout=5)
df = res._current_rows

In [17]:
df.sort_values(['exchange','pair'], inplace=True)
df.head(20) # ts contains last timestamps

,exchange,pair,ts,id,amount,insert_date,price,side,type
83,binance,ADA/BTC,2018-07-27 12:38:55.126,12874644,504.00000,2018-07-27 12:38:56.350,0.000020,buy,None
125,binance,ADA/ETH,2018-07-27 12:38:52.924,5426284,1643.00000,2018-07-27 12:38:57.378,0.000344,buy,None
219,binance,ADA/USDT,2018-07-27 12:38:57.303,3867368,762.00000,2018-07-27 12:38:58.453,0.158400,buy,None
258,binance,BCH/BTC,2018-07-27 12:38:32.832,10278622,0.58500,2018-07-27 12:38:59.948,0.100271,sell,None
176,binance,BCH/ETH,2018-07-27 12:33:37.285,1713582,0.09500,2018-07-27 12:34:23.898,1.722650,buy,None
283,binance,BCH/USDT,2018-07-27 12:38:48.843,6648478,0.04268,2018-07-27 12:39:01.373,791.660000,sell,None
114,binance,BTC/USDT,2018-07-27 12:39:02.311,52551259,0.00200,2018-07-27 12:39:03.055,7897.000000,buy,None
113,binance,BTG/BTC,2018-07-27 12:38:46.464,3241982,3.80000,2018-07-27 12:39:04.022,0.003654,buy,None
259,binance,BTG/ETH,2018-07-26 08:55:22.790,508894,1.18000,2018-07-27 12:25:15.011,0.064787,buy,None
288,binance,DASH/BTC,2018-07-27 12:38:58.722,3640119,1.92900,2018-07-27 12:39:06.096,0.030350,buy,None


In [18]:
# calculating timestamps for _cache parameter in Database and Markets classes
ts = [int(x/1e6) for x in df.ts.values.astype(np.uint64)]

In [19]:
ex_pair_ts = [(ex, pair, int(ts/1e6)) for ex, pair, ts in zip(df.exchange.values, df.pair.values, df.ts.values.astype(np.uint64))]

In [20]:
_cache = {}
for ex, p, ts in ex_pair_ts:
    if ex not in _cache:
        _cache[ex] = {}
    if p not in _cache[ex]:
        _cache[ex][p] = {}
    _cache[ex][p] = ts

In [22]:
_cache

{'binance': {'ADA/BTC': 1532695135126,
  'ADA/ETH': 1532695132924,
  'ADA/USDT': 1532695137303,
  'BCH/BTC': 1532695112832,
  'BCH/ETH': 1532694817285,
  'BCH/USDT': 1532695128843,
  'BTC/USDT': 1532695142311,
  'BTG/BTC': 1532695126464,
  'BTG/ETH': 1532595322790,
  'DASH/BTC': 1532695138721,
  'DASH/ETH': 1532695105017,
  'EOS/BTC': 1532695144073,
  'EOS/ETH': 1532695147223,
  'EOS/USDT': 1532695148959,
  'ETC/BTC': 1532695150429,
  'ETC/ETH': 1532695026207,
  'ETC/USDT': 1532695139375,
  'ETH/BTC': 1532695153859,
  'ETH/USDT': 1532695153768,
  'ICX/BTC': 1532695155993,
  'ICX/ETH': 1532695048304,
  'ICX/USDT': 1532695158599,
  'LTC/BTC': 1532695143092,
  'LTC/ETH': 1532695052717,
  'LTC/USDT': 1532695160119,
  'NEO/BTC': 1532695162158,
  'NEO/ETH': 1532695133947,
  'NEO/USDT': 1532695164585,
  'OMG/BTC': 1532695149971,
  'OMG/ETH': 1532694942487,
  'SC/BTC': 1532695083672,
  'SC/ETH': 1532694414009,
  'TRX/BTC': 1532695157913,
  'TRX/ETH': 1532695111887,
  'TRX/USDT': 1532695159709,

# Fetch transaction history from exchanges

In [23]:
exchanges = df_exchangs.id.tolist()
exchanges

['kraken',
 'poloniex',
 'yobit',
 'binance',
 'okex',
 'bittrex',
 'exmo',
 'hitbtc2',
 'cryptopia']

In [24]:
pair = "ETH/USD"
pair_alt = "ETH/USDT" # alternative representation
history = {}

# loading pairs from exchanges
for exchange in exchanges:
    history[exchange] = markets.fetch_trades(exchange, pair, limit=100)
    if history[exchange] == None:
        history[exchange] = markets.fetch_trades(exchange, pair_alt, limit=100)
    elif history[exchange] == None:
        del(history[exchange])

Error in D:\Git\Arbitron\markets\markets.py.fetch_trades(). poloniex No market symbol ETH/USD
Error in D:\Git\Arbitron\markets\markets.py.fetch_trades(). binance No market symbol ETH/USD
Error in D:\Git\Arbitron\markets\markets.py.fetch_trades(). hitbtc2 No market symbol ETH/USD
Error in D:\Git\Arbitron\markets\markets.py.fetch_trades(). cryptopia No market symbol ETH/USD


In [26]:
#history

## Kraken and Cryptopia have no IDs so we need to generate it by ourselves

In [27]:
infos = {}
for ex in exchanges:
    infos[ex] = history[ex][0]['id']
    
pprint(infos)

{'binance': '29302033',
 'bittrex': '1716163',
 'cryptopia': None,
 'exmo': '65410407',
 'hitbtc2': '337451377',
 'kraken': None,
 'okex': '1176971049600011',
 'poloniex': '10137814',
 'yobit': '201219590'}


## Check whether an Exchange have full info on transaction side/type

In [28]:
for ex in exchanges:
    print("\n============\n  " + ex + "\n============")
    pprint(history[ex][0])


  kraken
{'amount': 0.033,
 'cost': 15.34533,
 'datetime': '2018-07-27T09:02:05.126Z',
 'fee': None,
 'id': None,
 'info': ['465.01000', '0.03300000', 1532682125.1268, 's', 'l', ''],
 'order': None,
 'price': 465.01,
 'side': 'sell',
 'symbol': 'ETH/USD',
 'timestamp': 1532682125126,
 'type': 'limit'}

  poloniex
{'amount': 8.0,
 'cost': 3714.860488,
 'datetime': '2018-07-27T08:45:03.000Z',
 'fee': None,
 'id': '10137814',
 'info': {'amount': '8.00000000',
          'date': '2018-07-27 08:45:03',
          'globalTradeID': 384275420,
          'rate': '464.35756100',
          'total': '3714.86048800',
          'tradeID': 10137814,
          'type': 'sell'},
 'order': None,
 'price': 464.357561,
 'side': 'sell',
 'symbol': 'ETH/USDT',
 'timestamp': 1532681103000,
 'type': 'limit'}

  yobit
{'amount': 0.00241744,
 'datetime': '2018-07-27T11:29:32.000Z',
 'fee': {'cost': 4.83e-06, 'currency': 'ETH', 'rate': 0.002, 'type': 'taker'},
 'id': '201219590',
 'info': {'amount': 0.00241744,
  

# Compare prices

In [29]:
prices = [{'exchange':ex, 'price':history[ex][-1]['price']} for ex in exchanges]

In [30]:
df = pd.DataFrame(prices)
df

,exchange,price
0,kraken,463.770000
1,poloniex,464.050000
2,yobit,461.764091
3,binance,460.670000
4,okex,464.359000
5,bittrex,458.312000
6,exmo,460.000000
7,hitbtc2,461.220000
8,cryptopia,470.000000


In [31]:
# maximum spread
df_minmax = df.iloc[[df.price.idxmin(),df.price.idxmax()]]
df_minmax

,exchange,price
5,bittrex,458.312
8,cryptopia,470.000


In [32]:
# calculate diff and percentage
print(f"Difference between min and max = "
      f"{df_minmax.iloc[1].price - df_minmax.iloc[0].price} {pair.split('/')[1]}"
      f" or {round((df_minmax.iloc[1].price / df_minmax.iloc[0].price - 1)*100, 2)}%")

Difference between min and max = 11.687999999999988 USD or 2.55%
